In [1]:
import numpy as np
import nibabel as nib
import glob
import os
import re
import pandas as pd
import warnings
import json

In [2]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
data_dir = '/U-Net/data/'
predictions_dir = '/U-Net/predictions/'
N_LEVELS = 10

In [4]:
def add_uncertainty_class(predictions, uncertainty):
    predictions = np.mean(predictions,axis = 0)
        
    return np.where(predictions % 1 == 0, predictions, uncertainty)
       

In [5]:
def natural_keys(text):
    return [ int(c) if c.isdigit() else c for c in re.split(r'(\d+)', text) ]

In [6]:
def count_subfolders(path):
    return sum(os.path.isdir(os.path.join(path, entry)) for entry in os.listdir(path))
# print(count_subfolders(os.path.join(predictions_dir,"BRAIN_TUMOR_TASK1","UNCERTAINTY_CLASS","ensemble")))

In [7]:
def QDICE(pred, target, num_bins=10):
   
    bin_edges_low = [        i / num_bins for i in range(num_bins)]
    bin_edges_high = [edge + 1 / num_bins for edge in bin_edges_low]
    bin_edges_high[-1] += 1e-5  # small nudge to include 1.0
        
    dice_scores = []
    for low, high in zip(bin_edges_low, bin_edges_high):
        
        pred_band = ((pred >= low) & (pred < high))
        target_band = ((target >= low) & (target < high))

        intersection = np.sum(pred_band * target_band)
        union = np.sum(pred_band) + np.sum(target_band)

        if union > 0:
            dice = 2 * intersection / union
            dice_scores.append(dice)

    return np.mean(dice_scores) if dice_scores else 1.0

In [8]:
def DICE(pred, target, num_thresholds=10):
    thresholds = [i/num_thresholds for i in range(1,num_thresholds)]
    
    dice_scores = []

    for t in thresholds:
        pred_bin = (pred >= t).astype(np.uint8)
        target_bin = (target >= t).astype(np.uint8)
        
        intersection = np.sum(pred_bin * target_bin)
        union = np.sum(pred_bin) + np.sum(target_bin)

        if union > 0:
            dice = 2 * intersection / union
            dice_scores.append(dice)

    return np.mean(dice_scores) if dice_scores else 1.0

In [9]:
def GDC(pred, target):
    return 2*np.sum(np.sqrt(pred*target))/(np.sum(pred) + np.sum(target) + 1e-8)

In [10]:
def NORMAL_DICE(pred, target):
    return 2*np.sum(pred*target)/(np.sum(pred) + np.sum(target) + 1e-8)

In [11]:
def IOU(pred, target):
    intersection = np.sum(pred * target)
    union = np.sum((pred + target) > 0)
    
    return 0.0 if union == 0 else 1.0 - intersection / union

In [12]:
def GED(pred, target):
    
    # Intra-reference IoU
    ref_iou = [IOU(target[i], target[j]) for i in range(len(target)) for j in range(i + 1, len(target))]
    mean_ref_iou = np.mean(ref_iou) if ref_iou else 0

    # Intra-prediction IoU
    pred_iou = [IOU(pred[i], pred[j]) for i in range(len(pred)) for j in range(i + 1, len(pred))]
    mean_pred_iou = np.mean(pred_iou) if pred_iou else 0

    # Cross IoU (prediction vs reference)
    cross_iou = [IOU(tar, ref) for tar in target for ref in pred]
    
    mean_cross_iou = np.mean(cross_iou) if cross_iou else 0
    
    return 2 * mean_cross_iou - mean_pred_iou - mean_ref_iou

In [13]:
def IOU_2(pred, target):
    intersection = np.sum(pred * target)
    union = np.sum((pred + target) > 0)
    
    return 1.0 if union == 0 else 1.0 - intersection / union

def GED_2(pred, target):
    
    # Intra-reference IoU
    ref_iou = [IOU_2(target[i], target[j]) for i in range(len(target)) for j in range(i + 1, len(target))]
    mean_ref_iou = np.mean(ref_iou) if ref_iou else 0

    # Intra-prediction IoU
    pred_iou = [IOU_2(pred[i], pred[j]) for i in range(len(pred)) for j in range(i + 1, len(pred))]
    mean_pred_iou = np.mean(pred_iou) if pred_iou else 0

    # Cross IoU (prediction vs reference)
    cross_iou = [IOU_2(tar, ref) for tar in target for ref in pred]
    
    mean_cross_iou = np.mean(cross_iou) if cross_iou else 0
    
    return 2 * mean_cross_iou - mean_pred_iou - mean_ref_iou

In [14]:
def reading_ref_images(dataset,uncertainty_type):
    ref_data_dir = os.path.join(data_dir,dataset,uncertainty_type,'labelsTs')
    fnames = [os.path.basename(f) for f in glob.glob(ref_data_dir + '/*.nii.gz')]
    cases  = set([f.rsplit('_',1)[0] for f in fnames])
    return fnames, cases, ref_data_dir

In [15]:
uncertainty_types = ['SINGLE', 'MULTIPLE','UNCERTAINTY_CLASS']
datasets = [
    'KIDNEY'  ,  'BRAIN_TUMOR_TASK1','HEART', 'SIJ', 'LUNG', 'PROSTATE_TASK1', 'PROSTATE_TASK2', 
    'KNEE',    'BRAIN_GROWTH', 'SIJ', 'LUNG' , 'PANCREAS'
]
models = ['ensemble','hier_probabilistic',  'bayesian',  'probabilistic']

folds = ['fold0','fold1','fold2','fold3','fold4']

classes = {
    'BRAIN_TUMOR_TASK1':["TASK1","UNCERTAINTY"], 'KIDNEY':["KIDNEY","UNCERTAINTY"], 'KNEE':["KNEE","UNCERTAINTY"], 
    'PROSTATE_TASK1':["TASK1","UNCERTAINTY"], 'PROSTATE_TASK2':["TASK2","UNCERTAINTY"], 'BRAIN_GROWTH':["BRAIN_GROWTH","UNCERTAINTY"], 
    'PANCREAS':["PANCREAS","UNCERTAINTY"], 'SIJ':["SIJ","UNCERTAINTY"], 'LUNG':["LUNG","UNCERTAINTY"], 'HEART':["LUNGS","HEART","UNCERTAINTY"]
}
uncertainty_types_label = {
    'SINGLE':"SIN", 
    'MULTIPLE':"MUL",
    'UNCERTAINTY_CLASS':"DIR"
}

In [16]:
# !!!!!!!!!!!!!!!!!!!!!!!! WRONG WRONG WRONG: <<<it creates dublicate values>>>
results = []
for dataset in datasets:
    for model in models:
        for uncertainty_type in uncertainty_types:
            
            if uncertainty_type=="UNCERTAINTY_CLASS" and model!="ensemble":
                continue
                
            
            # return the list of ref images name and the path to the reference images directory
            ref_fname, ref_cases,ref_dir_path = reading_ref_images(dataset,uncertainty_type)
            
            # the path to the fold directories containing the predictions
            eval_dirs_path = [os.path.join(predictions_dir,dataset,uncertainty_type,model,fold) for fold in folds]

            for ncase, case in enumerate(ref_cases): # loop over the ref cases
                     
                # Load all reference images
                refs = [nib.load(fname).get_fdata() for fname in glob.glob(os.path.join(ref_dir_path, f'{case}_*.nii.gz'))]

                # Load predictions for each case seperated by fold (list items)
                evals = evals_un_mul = evals_un_sin = []
                
                model_dir_path = os.path.join(predictions_dir,dataset,uncertainty_type,model)
                # if uncertainty_type=="UNCERTAINTY_CLASS" and count_subfolders(model_dir_path)<5:
                if uncertainty_type=="UNCERTAINTY_CLASS": 
                    # if count_subfolders(model_dir_path)<5:
                    evals.extend([nib.load(fname).get_fdata() for fname in glob.glob(os.path.join(model_dir_path, f'{case}_*.nii.gz'))])
                    evals_un_mul = [nib.load(os.path.join(predictions_dir,dataset,"MULTIPLE",model,'fold' + str(i),case+'_0.nii.gz')).get_fdata() for i in range(5)]
                    
                    evals_un_sin = [nib.load(os.path.join(predictions_dir,dataset,"SINGLE",model,'fold' + str(i),case+'_0.nii.gz')).get_fdata() for i in range(5)]
                    
                else:
                    for eval_dir in eval_dirs_path:
                        evals.extend([nib.load(fname).get_fdata() for fname in glob.glob(os.path.join(eval_dir, f'{case}_*.nii.gz'))])
                    
                classes_object = len(classes[dataset]) + (1 if uncertainty_type=="UNCERTAINTY_CLASS" else 0)
                    
                # Binarize the predictions; for example, if the image has 3 classes, the binarized images will be 3 images with values 0 and 1.
                # for each class, the binarized image will have 1 for the pixels that belong to that class and 0 for the rest.
                
                # print(ncase,dataset,model,uncertainty_type,case,len(results))
                for label in range(1, classes_object):
                    
                    binarized_refs = [(item == label).astype(np.float32) for item in refs] # binarize the reference images.
                    binarized_evals = [(item == label).astype(np.float32) for item in evals] # binarize the prediction images; all folds.
                    avg_ref = np.mean(binarized_refs, axis=0)
                    avg_eval = np.mean(binarized_evals, axis=0)                
                    
                    # Add the results of all the metrics to the results list
                    # Only GED function accepts list of images as input the rest of the functions accept only one image as input.
                    results +=[
                        # (case,dataset, classes[dataset][label-1],uncertainty_types_label[uncertainty_type], model, QDICE(avg_ref, avg_eval, N_LEVELS),"qdice"),
                        # (case,dataset, classes[dataset][label-1],uncertainty_types_label[uncertainty_type], model, DICE(avg_ref, avg_eval, N_LEVELS),"dice"),
                        # (case,dataset, classes[dataset][label-1],uncertainty_types_label[uncertainty_type], model, GDC(avg_ref, avg_eval),"gdc"),
                        (case,dataset, classes[dataset][label-1],uncertainty_types_label[uncertainty_type], model, NORMAL_DICE(avg_ref, avg_eval),"ndice"),
                        
                        # (case,dataset, classes[dataset][label-1],uncertainty_types_label[uncertainty_type], model, GED(binarized_refs, binarized_evals),"ged"),
                        # (case,dataset, classes[dataset][label-1],uncertainty_types_label[uncertainty_type], model, GED_2(binarized_refs, binarized_evals),"ged2")
                        
                    ]  
                    
                    
                    if uncertainty_type=="UNCERTAINTY_CLASS":                        
                                                
                        
                        avg_eval = add_uncertainty_class(evals_un_sin,len(classes[dataset]))
                        results +=[
                            # (case,dataset, classes[dataset][label-1],"DSI", model, QDICE(avg_ref, avg_eval, N_LEVELS),"qdice"),
                            # (case,dataset, classes[dataset][label-1],"DSI", model, DICE(avg_ref, avg_eval, N_LEVELS),"dice"),
                            # (case,dataset, classes[dataset][label-1],"DSI", model, GDC(avg_ref, avg_eval),"gdc"),
                            (case,dataset, classes[dataset][label-1],"DSI", model, NORMAL_DICE(avg_ref, np.where(avg_eval == label, 1, 0)),"ndice"),
                            # (case,dataset, classes[dataset][label-1],"DSI", model, GED(binarized_refs, binarized_evals),"ged"),
                            # (case,dataset, classes[dataset][label-1],"DSI", model, GED_2(binarized_refs, binarized_evals),"ged2")
                        ]
                        
                        # print("DSI",NORMAL_DICE(avg_ref, np.where(avg_eval == label, 1, 0)))
                        
                        
                        
                        avg_eval = add_uncertainty_class(evals_un_mul,len(classes[dataset]))
                        
                        results +=[
                            # (case,dataset, classes[dataset][label-1],"DMU", model, QDICE(avg_ref, avg_eval, N_LEVELS),"qdice"),
                            # (case,dataset, classes[dataset][label-1],"DMU", model, DICE(avg_ref, avg_eval, N_LEVELS),"dice"),
                            # (case,dataset, classes[dataset][label-1],"DMU", model, GDC(avg_ref, avg_eval),"gdc"),
                            (case,dataset, classes[dataset][label-1],"DMU", model, NORMAL_DICE(avg_ref, np.where(avg_eval == label, 1, 0)),"ndice"),
                            # (case,dataset, classes[dataset][label-1],"DMU", model, GED(binarized_refs, binarized_evals),"ged"),
                            # (case,dataset, classes[dataset][label-1],"DMU", model, GED_2(binarized_refs, binarized_evals),"ged2")
                        ]
                        # print("DMU",NORMAL_DICE(avg_ref, np.where(avg_eval == label, 1, 0)))
                        # print(
                        #     "DMU",np.unique(avg_ref,return_counts=True),
                        #     np.unique(np.where(avg_eval == label, 1, 0),return_counts=True)
                        # )
                        
                


In [17]:
FINAL_RESULTS = pd.DataFrame(columns=["id","dataset", "segment", "type", "model", "result", "metric"])
FINAL_RESULTS = pd.concat([FINAL_RESULTS,pd.DataFrame(results, columns=FINAL_RESULTS.columns)], ignore_index=True)
FINAL_RESULTS.to_csv("RESUITS_3.csv",index=False)


In [18]:
# FINAL_RESULTS.head(20)

In [19]:
# FINAL_RESULTS.pivot_table(
#     index=["dataset", "type", "model", "segment"],
#     columns="metric",
#     values="result",
#     aggfunc="mean"
# ).reset_index()

In [20]:
# FINAL_RESULTS.groupby(["dataset","type","model","segment","metric"]).agg({"result": ["mean", "std"]})

In [21]:
# FINAL_RESULTS.pivot_table(
#     index=["dataset", "type", "model", "segment"],
#     columns="metric",
#     values="result",
#     aggfunc="mean"
# ).reset_index()

In [22]:
# FINAL_RESULTS.loc[
#     (FINAL_RESULTS["dataset"] == "KIDNEY") & 
#     (FINAL_RESULTS["model"] == "ensemble") & 
#     (FINAL_RESULTS["metric"] == "qdice") & 
#     (FINAL_RESULTS["type"] == "SINGLE")
# ,["id","result"]]
